In [23]:
# Import packages
import pandas as pd
import numpy as np
import librosa
import librosa.display # Visualise Spectrogram
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from io import BytesIO

In [24]:
# Load dataset
df = pd.read_csv("./spotify_tracks_50.csv")
df.head()

,track_uri,album_name,album_uri,artist_name,artist_uri,duration_ms,pos,track_name,inside_playlists
0,spotify:track:4pLwZjInHj3SimIyN9SnOz,Dangerous Woman,spotify:album:4lVR2fg3DAUQpGVJ6DciHW,Ariana Grande,spotify:artist:66CXWjxzNUsdJxJ2JdwvnR,226160,0,Side To Side,[584466 290613 339107 ... 932309 757216 867665]
1,spotify:track:37f4ITSlgPX81ad2EvmVQr,Wildfire,spotify:album:0mFDIOqypzHp6Xd0el1hoT,Rachel Platten,spotify:artist:3QLIkT4rD2FMusaqmkepbq,204013,1,Fight Song,[584466 290613 540271 ... 535890 217688 26462]
2,spotify:track:6i0V12jOa3mr6uu4WYhUBr,Heathens,spotify:album:3J8W9AOjQhnBLCX33m3atT,Twenty One Pilots,spotify:artist:3YQKmKGau1PzlVlkL1iodx,195920,2,Heathens,[584466 290613 339107 ... 349806 168239 932309]
3,spotify:track:2DpCdPMg1BADE4HDnxt3Rd,"Sit Still, Look Pretty",spotify:album:2cE2eOy7alOZHpuelJEV8Q,Daya,spotify:artist:6Dd3NScHWwnW6obMFbl1BH,202226,3,"Sit Still, Look Pretty",[584466 414677 809633 ... 6555 349244 395752]
4,spotify:track:6Knv6wdA0luoMUuuoYi2i1,MY HOUSE,spotify:album:5lkNnHVlnCCCV304t89wOH,Flo Rida,spotify:artist:0jnsk9HBra6NMjO2oANoPY,192190,4,My House,[584466 290613 968716 ... 257996 349976 6463]


In [1]:
# Authenticate with Spotify
client_credentials_manager = SpotifyClientCredentials(client_id='cient_id', client_secret='client_secret')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

NameError: name 'SpotifyClientCredentials' is not defined

In [29]:
# Function to extract audio file
def get_audio_file(track_uri):
    # Fetch the track details from Spotify API
    track = sp.track(track_uri)

    print(track) # debugging line, remove when done
    
    # # Fetch the track's preview URL
    # audio_url = track['preview_url']
    
    # if audio_url:
    #     # Fetch the audio content
    #     response = requests.get(audio_url)
        
    #     if response.status_code == 200:
    #         audio_data = BytesIO(response.content)
    #         return audio_data
    #     else:
    #         print(f"Error downloading track with URI {track_uri}")
    #         return None
    # else:
    #     print(f"No preview available for track with URI {track_uri}")
    #     return None

# Function to extract audio features
def extract_audio_features(audio_data):
    # Load the audio file
    y, sr = librosa.load(audio_data, sr=None)
    
    # Extract Harmonic Content
    harmonic, _ = librosa.effects.hpss(y)
    harmonic_content = np.mean(librosa.feature.chroma_cqt(y=harmonic, sr=sr), axis=1)
    
    # Extract Frequency Distribution (MFCCs)
    mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13).T, axis=0)
    
    # Extract Spectrogram Data
    spectrogram = np.mean(librosa.feature.melspectrogram(y=y, sr=sr).T, axis=0)
    
    return harmonic_content, mfccs, spectrogram

In [30]:
# Extract audio features for all songs
# Initialize lists to store features
harmonic_list = []
mfccs_list = []
spectrogram_list = []

# Loop through all tracks
for track_uri in df['track_uri']:
    track_id = track_uri.split(":")[-1]
    audio_data = get_audio_file(track_id)
    if audio_data:
        harmonic, mfccs, spectrogram = extract_audio_features(audio_data)
        harmonic_list.append(harmonic)
        mfccs_list.append(mfccs)
        spectrogram_list.append(spectrogram)
    else:
        continue

# Convert lists to numpy arrays
harmonic_features = np.array(harmonic_list)
mfccs_features = np.array(mfccs_list)
spectrogram_features = np.array(spectrogram_list)

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/66CXWjxzNUsdJxJ2JdwvnR'}, 'href': 'https://api.spotify.com/v1/artists/66CXWjxzNUsdJxJ2JdwvnR', 'id': '66CXWjxzNUsdJxJ2JdwvnR', 'name': 'Ariana Grande', 'type': 'artist', 'uri': 'spotify:artist:66CXWjxzNUsdJxJ2JdwvnR'}], 'available_markets': [], 'external_urls': {'spotify': 'https://open.spotify.com/album/4lVR2fg3DAUQpGVJ6DciHW'}, 'href': 'https://api.spotify.com/v1/albums/4lVR2fg3DAUQpGVJ6DciHW', 'id': '4lVR2fg3DAUQpGVJ6DciHW', 'images': [{'url': 'https://i.scdn.co/image/ab67616d0000b2737227f5854239a059af211d5b', 'width': 640, 'height': 640}, {'url': 'https://i.scdn.co/image/ab67616d00001e027227f5854239a059af211d5b', 'width': 300, 'height': 300}, {'url': 'https://i.scdn.co/image/ab67616d000048517227f5854239a059af211d5b', 'width': 64, 'height': 64}], 'name': 'Dangerous Woman', 'release_date': '2016-05-20', 'release_date_precision': 'day', 'total_tracks': 15, 'type': 'album', 'uri